In [7]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

# --- データベースの準備 ---
# ファイル名
db_name = 'google_repos.db'

# データベースにつなぐ
conn = sqlite3.connect(db_name)
cur = conn.cursor()

# 失敗したときにやり直せるように、前のテーブルを消す
cur.execute('DROP TABLE IF EXISTS repositories')

# テーブルを作る
cur.execute('''
    CREATE TABLE repositories (
        name TEXT,
        language TEXT,
        stars TEXT
    )
''')
conn.commit()


# --- スクレイピング開始 ---

# GoogleのGitHubトップページ（ここに固定されたリポジトリがある）
url = 'https://github.com/google'

# ブラウザのふりをする
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

print("サイトにアクセスします...")
response = requests.get(url, headers=headers)

# ステータスコードが200（成功）のときだけ処理する
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # ページのタイトルを表示してみる
    print("タイトル:", soup.title.text)

    # 検証ツールで調べたら、固定されたリポジトリは 'pinned-item-list-item-content' というクラスだった
    repo_list = soup.find_all('div', class_='pinned-item-list-item-content')
    
    print(len(repo_list), "件見つかりました。")

    # 1つずつ取り出す
    for repo in repo_list:
        
        # 1. 名前を取る
        # spanタグの class="repo" に名前が入ってる
        name_tag = repo.find('span', class_='repo')
        if name_tag:
            repo_name = name_tag.text.strip()
        else:
            repo_name = "名前なし"

        # 2. 言語を取る
        # itemprop="programmingLanguage" という属性を探す
        lang_tag = repo.find('span', itemprop='programmingLanguage')
        if lang_tag:
            lang_name = lang_tag.text.strip()
        else:
            lang_name = "言語なし"

        # 3. スター数を取る
        # リンク（aタグ）の中にスター数があるけど、リンクがたくさんあるので
        # href（リンク先）に "stargazers" という文字が入ってるものを探す
        star_num = "0"
        all_links = repo.find_all('a')
        
        for link in all_links:
            link_url = link.get('href')
            # リンクURLがあって、その中に stargazers が含まれていたら
            if link_url and 'stargazers' in link_url:
                star_num = link.text.strip()
                # カンマを消す
                star_num = star_num.replace(',', '')
                # 見つかったらループを終わる
                break

        # 画面に表示
        print(repo_name, "/", lang_name, "/", star_num)

        # データベースに入れる
        cur.execute('INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)', (repo_name, lang_name, star_num))
        
        # サーバーに負荷をかけないように1秒待つ（重要）
        time.sleep(1)

    # 最後に保存する
    conn.commit()
    print("データベースへの保存が終わりました。")

else:
    print("アクセスに失敗しました。")


# --- 保存できたか確認する ---
print("\n--- データベースの中身 ---")

# 全部のデータを取ってくる
cur.execute('SELECT * FROM repositories')
data_list = cur.fetchall()

for data in data_list:
    print(data)

# データベースを閉じる
conn.close()

サイトにアクセスします...
タイトル: Google · GitHub
6 件見つかりました。
material-design-icons / 言語なし / 52.6k
guava / Java / 51.3k
zx / JavaScript / 44.9k
styleguide / HTML / 38.7k
leveldb / C++ / 38.4k
googletest / C++ / 37.5k
データベースへの保存が終わりました。

--- データベースの中身 ---
('material-design-icons', '言語なし', '52.6k')
('guava', 'Java', '51.3k')
('zx', 'JavaScript', '44.9k')
('styleguide', 'HTML', '38.7k')
('leveldb', 'C++', '38.4k')
('googletest', 'C++', '37.5k')
